# 实验说明

## 作业说明

### 目标：

训练一个玩2048的神经网络，并得到较高的准确率。

### 背景：

2048是一个益智小游戏，规则为：控制所有方块向同一个方向运动，两个相同数字方块撞在一起后合并，成为他们的和。每次操作时会随机生成一个2或者4，最终得到一个“2048”的方块就算胜利。规则的直观解释：[Click to Play 2048](https://play2048.co/)

本教程展示如何训练一个玩2048的神经网络模型，并评估其最终能够得到的分数。

#### 建模过程：

2048游戏可以理解为一个这样的过程：

<blockquote>
    
有一个**局面（state）**，4x4格子上的一些数字。
    
<img src="https://data.megengine.org.cn/megstudio/images/2048_demo.png" width=256 height=256 />

你可以选择做一些**动作（action）**，比如按键盘上的上/下/左/右键。

你有一些**策略（policy）**，比如你觉得现在按左最好，因为这样有两个8可以合并。对于每个动作，可以用一个打分函数来决定你的策略。

在按照策略做完动作之后，你会得到一个**奖励（reward）**，比如因为两个8合并，分数增加了16，这个16可以被看作是这一步的奖励。

在许多步之后，游戏结束，你会得到一个**回报（return）**，即游戏的最终分数。

</blockquote>

由此，我们将2048建模为一个马尔可夫决策过程，其求解可以通过各种强化学习方法来完成。在baseline中，我们使用了 [Double DQN](https://arxiv.org/abs/1509.06461)。

### 任务：

Q1：训练模型

运行baseline，训练和评估模型。观察游戏结束时的滑动平均分数。你可以调用`print_grid`函数输出模型玩游戏的过程，以判断模型是否可以得到合理的结果。
提供参考数据：纯随机游玩，平均分数约为570分。在baseline的训练过程中，模型最高可以达到8000分，平均为2000分左右。

请你修改参数，模型结构等，使得游戏的平均分数尽可能地高。请注意：这里的平均分数指每个游戏结束**最终分数**的平均值。
**请于q1.diff提交你的代码。**

## 数据集

2048游戏代码来源：[console2048](https://github.com/Mekire/console-2048/blob/master/console2048.py)

## 文件存储
实验中生成的文件可以存储于 workspace 目录中。 查看工作区文件，该目录下的变更将会持久保存。 您的所有项目将共享一个存储空间，请及时清理不必要的文件，避免加载过慢。

## 实验步骤

1.导入库

In [ ]:
import numpy as np 
from torch.optim import Adam
from random import randint, shuffle

from tqdm import tqdm
import torch.nn as nn
import random
import torch
import torch.tensor as tensor
import torch.functional as F
from torch.utils.data import Dataset, DataLoader
from matplotlib import pyplot as plt

In [ ]:
use_cuda = True

2.2048游戏函数

In [ ]:
# https://github.com/Mekire/console-2048/blob/master/console2048.py

def push_left(grid):
    moved, score = False, 0
    rows, columns = grid.shape[0], grid.shape[1]
    for k in range(rows):
        i, last = 0, 0
        for j in range(columns):
            e = grid[k, j]
            if e:
                if e == last:
                    grid[k, i-1]+=e
                    score += e
                    last, moved = 0, True
                else:
                    moved |= (i != j)
                    last=grid[k, i]=e
                    i+=1
        while i<columns:
            grid[k,i]=0
            i+=1
    return score if moved else -1


def push_right(grid):
    moved, score = False, 0
    rows, columns = grid.shape[0], grid.shape[1]
    for k in range(rows):
        i = columns-1
        last  = 0
        for j in range(columns-1,-1,-1):
            e = grid[k, j]
            if e:
                if e == last:
                    grid[k, i+1]+=e
                    score += e
                    last, moved = 0, True
                else:
                    moved |= (i != j)
                    last=grid[k, i]=e
                    i-=1
        while 0<=i:
            grid[k, i]=0
            i-=1
    return score if moved else -1


def push_up(grid):
    moved, score = False, 0
    rows, columns = grid.shape[0], grid.shape[1]
    for k in range(columns):
        i, last = 0, 0
        for j in range(rows):
            e = grid[j, k]
            if e:
                if e == last:
                    score += e
                    grid[i-1, k]+=e
                    last, moved = 0, True
                else:
                    moved |= (i != j)
                    last=grid[i, k]=e
                    i+=1
        while i<rows:
            grid[i, k]=0
            i+=1
    return score if moved else -1


def push_down(grid):
    moved, score = False, 0
    rows, columns = grid.shape[0], grid.shape[1]
    for k in range(columns):
        i, last = rows-1, 0
        for j in range(rows-1,-1,-1):
            e = grid[j, k]
            if e:
                if e == last:
                    score += e
                    grid[i+1, k]+=e
                    last, moved = 0, True
                else:
                    moved |= (i != j)
                    last=grid[i, k]=e
                    i-=1
        while 0<=i:
            grid[i, k]=0
            i-=1
    return score if moved else -1


def push(grid, direction):
    if direction&1:
        if direction&2:
            score = push_down(grid)
        else:
            score = push_up(grid)
    else:
        if direction&2:
            score = push_right(grid)
        else:
            score = push_left(grid)
    return score


def put_new_cell(grid):
    n = 0
    r = 0
    i_s=[0]*16
    j_s=[0]*16
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            if not grid[i,j]:
                i_s[n]=i
                j_s[n]=j
                n+=1
    if n > 0:
        r = randint(0, n-1)
        grid[i_s[r], j_s[r]] = 2 if random.random() < 0.9 else 4
    return n

def any_possible_moves(grid):
    """Return True if there are any legal moves, and False otherwise."""
    rows = grid.shape[0]
    columns = grid.shape[1]
    for i in range(rows):
        for j in range(columns):
            e = grid[i, j]
            if not e:
                return True
            if j and e == grid[i, j-1]:
                return True
            if i and e == grid[i-1, j]:
                return True
    return False



def prepare_next_turn(grid):
    """
    Spawn a new number on the grid; then return the result of
    any_possible_moves after this change has been made.
    """
    empties = put_new_cell(grid)    
    return empties>1 or any_possible_moves(grid)


def print_grid(grid_array):
    """Print a pretty grid to the screen."""
    print("")
    wall = "+------"*grid_array.shape[1]+"+"
    print(wall)
    for i in range(grid_array.shape[0]):
        meat = "|".join("{:^6}".format(grid_array[i,j]) for j in range(grid_array.shape[1]))
        print("|{}|".format(meat))
        print(wall)


class Game:
    def __init__(self, cols=4, rows=4):
        self.grid_array = np.zeros(shape=(rows, cols), dtype='uint16')
        self.grid = self.grid_array
        for i in range(2):
            put_new_cell(self.grid)
        self.score = 0
        self.end = False

    def copy(self):
        rtn = Game(self.grid.shape[0], self.grid.shape[1])
        for i in range(self.grid.shape[0]):
            for j in range(self.grid.shape[1]):
                rtn.grid[i,j]=self.grid[i,j]
        rtn.score = self.score
        rtn.end = self.end
        return rtn

    def max(self):
        m = 0
        for i in range(self.grid.shape[0]):
            for j in range(self.grid.shape[1]):
                if self.grid[i,j]>m:
                    m = self.grid[i,j]
        return m

    def move(self, direction):
        if direction&1:
            if direction&2:
                score = push_down(self.grid) # 3
            else:
                score = push_up(self.grid) # 1
        else:
            if direction&2:
                score = push_right(self.grid) # 2
            else:
                score = push_left(self.grid) # 0
        if score == -1:
            return 0
        self.score += score
        if not prepare_next_turn(self.grid):
            self.end = True
        return 1

    def display(self):
        print_grid(self.grid_array)

def random_play(game):
    moves = [0,1,2,3]
    while not game.end:
        shuffle(moves)
        for m in moves:
            if game.move(m):
                break
    return game.score

3.定义记忆回放类并实例化

在记录一次决策过程后，我们存储到该类中，并在训练时选择一部分记忆进行训练。

In [ ]:
class MyDataset(Dataset):
    def __init__(self, path):
        super(MyDataset, self).__init__()
        data = np.load(path)
        self.states = data['states']
        self.actions = data['actions']

    def __getitem__(self, index):
        state = self.states[index].copy()
        for i in range(16):
            state[i] = (1 << state[i]) & -2
        state = state.reshape(4, 4)
        action = self.actions[index]
        if action == 'up':
            action = 1
        elif action == 'down':
            action = 3
        elif action == 'left':
            action = 0
        elif action == 'right':
            action = 2
        else:
            raise ValueError()
        return make_input(state), action

    def __len__(self):
        return self.states.shape[0]


batch_size = 512
dataset = MyDataset('trainset_large.npz')
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

4.定义模型结构

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv00 = nn.Conv2d(16, 256, kernel_size=(2, 1), stride=1, padding=0)
        self.relu00 = nn.ReLU()
        self.conv01 = nn.Conv2d(16, 256, kernel_size=(1, 2), stride=1, padding=0)
        self.relu01 = nn.ReLU()

        self.conv10 = nn.Conv2d(256, 256, kernel_size=(2, 1), stride=1, padding=0)
        self.relu10 = nn.ReLU()
        self.conv11 = nn.Conv2d(256, 256, kernel_size=(1, 2), stride=1, padding=0)
        self.relu11 = nn.ReLU()

        self.conv20 = nn.Conv2d(256, 256, kernel_size=(2, 1), stride=1, padding=0)
        self.relu20 = nn.ReLU()
        self.conv21 = nn.Conv2d(256, 256, kernel_size=(1, 2), stride=1, padding=0)
        self.relu21 = nn.ReLU()

        self.fc1 = nn.Linear(8704, 16)
        self.reluu = nn.ReLU()
        self.fc2 = nn.Linear(16, 4)

    def forward(self, x):
        x1 = self.relu00(self.conv00(x))
        x2 = self.relu01(self.conv01(x))

        x3 = torch.flatten(self.relu10(self.conv10(x1)), 1)
        x4 = torch.flatten(self.relu11(self.conv11(x1)), 1)

        x5 = torch.flatten(self.relu20(self.conv20(x2)), 1)
        x6 = torch.flatten(self.relu21(self.conv21(x2)), 1)
        
        xx = torch.cat([x3, x4, x5, x6], 1)
        xx = self.reluu(self.fc1(xx))
        xx = self.fc2(xx)
        return xx.reshape((-1, 4))

    
model = Net()
model_target = Net()

In [ ]:
"""
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(16 * 4 * 4, 4096)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(4096, 8192)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(8192, 8192)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(8192, 16)
        self.relu4 = nn.ReLU()
        self.fc5 = nn.Linear(16, 4)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        x = self.relu4(x)

        x = self.fc5(x)
        return x


model = Net()
model_target = Net()
"""

5.定义输入转化函数，使得局面可以被输入进模型。

In [ ]:
table = {2**i: i for i in range(1, 16)}
table[0] = 0

rev_table = {i: 2**i for i in range(1, 16)}
rev_table[0] = 0


def make_input(grid):
    g0 = grid
    r = np.zeros(shape=(16, 4, 4), dtype=np.float32)
    for i in range(4):
        for j in range(4):
            v = g0[i, j].item()
            r[table[v], i, j] = 1
    return r


def convert_from_input(inp):
    inp = F.argmax(inp, axis=0)
    inp = inp.detach().numpy()
    grid = np.zeros(shape=(4, 4), dtype='int16')
    for i in range(4):
        for j in range(4):
            grid[i][j] = rev_table[inp[i][j]]
    return grid

6.定义优化器

In [ ]:
opt = Adam(model.parameters(), lr=1e-2)
loss_func = nn.CrossEntropyLoss()

In [ ]:
test_time = 200
avg_score = 0
max_score = 0

def play_a_game():
    global avg_score, max_score
    game = Game()
    while True:
        if not any_possible_moves(game.grid):
            avg_score += game.score
            max_score = max(max_score, game.score)
            break

        tmp = make_input(game.grid)
        status = tensor(tmp).unsqueeze(0)
        if use_cuda:
            status = status.cuda()

        a = torch.argmax(model(status).detach(), 1)
        a = a.item()

        pre_grid = game.grid.copy()
        game.move(a)
        while (game.grid == pre_grid).all():
            a = (a + 1) % 4
            game.move(a)


def evaluate():
    with tqdm(total=test_time, desc="test_time") as tq:
        for i in range(test_time):
            play_a_game()
            tq.set_postfix(
                {
                    "avg_score": "{0:1.5f}".format(avg_score / (i + 1)),
                    "max_score": "{0:1.5f}".format(max_score)
                }
            )
            tq.update(1)
    return avg_score / (test_time + 1)

# evaluate()

7.模型训练

In [ ]:
epochs = 50
if use_cuda:
    model = model.cuda()
accs = []
scs = []

def train():
    batch_num = len(dataloader)
    max_avg = 0
    for epo in range(epochs):
        acc_tot = 0
        tot = 0
        with tqdm(total=batch_num, desc="batch_num") as tq:
            for ind, data in enumerate(dataloader):
                x, y = data[0], data[1]
                if use_cuda:
                    x, y = x.cuda(), y.cuda()
                y_pred = model(x)
                loss = loss_func(y_pred, y)
                acc_tot += (torch.argmax(y_pred, 1) == y).sum().item()
                tot += y.shape[0]
                tq.set_postfix(
                {
                    "epoch": "{}".format(epo),
                    "loss": "{0:1.5f}".format(loss.item()),
                    "acc": "{0:1.5f}".format(acc_tot / tot)
                }
                )
                tq.update(1)

                opt.zero_grad()
                loss.backward()
                opt.step()
        if epo % 2 == 0:
            scs.append(evaluate())
            avg_score = max_score = 0
            if scs[-1] > max_avg:
                max_avg = scs[-1]
                torch.save(model.state_dict(), '1.ckpt')
        accs.append(acc_tot / tot)


#model.load_state_dict(torch.load('1.ckpt'))
train()
# plt.plot(accs)
plt.plot(accs)
plt.show()
torch.save(model.state_dict(), '1.ckpt')